In [20]:
# Code for accessing Ollama and posing questions
# Test wether it works to invoke a model and pose a question

MODEL = "gemma:2b"
# MODEL = "mistral"
# MODEL = "llama2"


from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

# Actual method to invoke a model and ask a question:
# model.invoke("Tell me a joke")

Get a list of paths of docx documents I want to load into memory

In [1]:
import os

data_path = os.getenv("DOCUMENTS_PATH")

# Retrieve the path names of files in the specified directory
docx_file_paths = [os.path.join(data_path, file) for file in os.listdir(data_path)]
docx_file_paths_cleaned = [item for item in docx_file_paths if item.endswith('.docx')]

**# Load in files to apply RAG to**
Load the docx files into memory using the docx-python library

In [2]:
from langchain_community.document_loaders import Docx2txtLoader

loaded_texts = [Docx2txtLoader(path) for path in docx_file_paths_cleaned]


# Create embedding
## test on one document text first, then try to iterate over all texts


In [8]:
test_pages = loaded_texts[5].load_and_split()

# Create Templete


In [3]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")
print(prompt.format(context="Here is some context", question="Here is a question"))



Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



# Create embeddings

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(test_pages, embedding = embeddings)

In [13]:
retriever = vectorstore.as_retriever()

retriever.invoke("Initiatiefnaam")


[Document(page_content='Op het moment dat de nieuwe persoonslijst van de niet-ingezetene op de bevragingenserver staat, kan die bevraagd worden door de geautoriseerde afnemers waaronder DigiD.\n\nDe doorlooptijd van dit proces bedraagt in de huidige configuratie vele uren, vanwege de vele onafhankelijke tijdsintervallen; te lang voor de niet-ingezetene om op te wachten bij inschrijving aan het RNI-loket. Wanneer de totale doorlooptijd vanaf inschrijven tot bevraagbaar zijn voor DigiD teruggebracht kan worden tot maximaal een kwartier, dan wordt het mogelijk om het inschrijven aan de RNI-balie en het aanvragen van DigiD in één loketbezoek af te handelen. Op dit moment zijn er drie DigiD-balies die ook RNI-loket zijn: Breda, Den Haag en Venlo. Het voornemen bestaat om het aanvragen van DigiD mogelijk te maken bij alle RNI-loketten.\n\n\n\nHet proces van het aanvragen van DigiD Buitenland verloopt op hoofdlijnen als volgt:\n\nDe aanvrager moet in het bezit zijn van een EER-nationaliteit e

# Create Chain

In [17]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
)


<bound method BaseModel.schema of <class 'langchain_core.runnables.base.RunnableSequence'>>


# Run the code

In [ ]:

#chain.invoke(
#    {
#
#    "question": "Geef de code van dit initiatief, deze heeft de vorm van AAA-11-11 " 
#    }
#)